In [ ]:
from badlands.model import Model as badlandsModel

In [ ]:
# 1. Cargar el modelo
model = badlandsModel()

In [ ]:
# 2. Definir el archivo XML (ajusta el nombre si está en otra carpeta)
model.load_xml('input_forced_out.xml')
model.run_to_time(20000000)

In [4]:
# transformar_uplift_precip.py
import numpy as np
import pandas as pd
import os

# --- Configuración ---
input_csv = "stepMapOff.csv"    # tu archivo con valores -100..15900 (puede tener 1 o más columnas)
dem_csv   = "xyz.csv"   # tu DEM original (x,y,elev) para asegurar tamaño/grilla
# Rango deseado (ajusta según tu calibración)
uplift_range = (0.00005, 0.0015)  # rango para uplift en m/año (ej: 0.05 mm/a -> 0.00005 m/a)

# --- Leer input robustamente ---
def read_any_csv(path):
    # intenta detectar separador automáticamente
    try:
        df = pd.read_csv(path, sep=None, engine="python", header=None, comment="#")
    except Exception:
        df = pd.read_csv(path, header=None)  # fallback
    return df

vals_df = read_any_csv(input_csv)
print("Input shape:", vals_df.shape)
# si viene en una sola columna con strings "100.0,-11119.0,-13495.0", separarlo:
if vals_df.shape[1] == 1 and vals_df.iloc[:,0].astype(str).str.contains(",").any():
    # intenta split por coma y reconstruir
    expanded = vals_df.iloc[:,0].str.split(",", expand=True)
    vals_df = expanded.apply(pd.to_numeric, errors="coerce")

# tomar la columna de interés: si hay >1 columna asumimos ultima = valor objetivo
values = vals_df.iloc[:, -1].astype(float).dropna().values
print("Valores válidos leídos:", len(values), "min,max = ", values.min(), values.max())

# --- Leer DEM para tamaño de grilla ---
dem = read_any_csv(dem_csv)
# si DEM tiene 3 columnas x,y,elev (header None) su número de celdas es:
if dem.shape[1] >= 3:
    n_cells_dem = dem.shape[0]
else:
    # si dem está en formato grilla (ncols,nrows) no estándar, asume flatten length
    n_cells_dem = dem.shape[0]
print("Celdas DEM:", n_cells_dem)

# --- Función de reescalado lineal ---
def rescale(arr, target_min, target_max):
    a = np.array(arr, dtype=float)
    a_min, a_max = np.nanmin(a), np.nanmax(a)
    if a_max == a_min:
        return np.full(a.shape, 0.5*(target_min+target_max))
    scaled = (a - a_min) / (a_max - a_min)
    return target_min + scaled * (target_max - target_min)

# --- Generar archivo de uplift (una columna) ---
# Opción A: si quieres derivar uplift a partir del gradiente del input (ej. si input es elev)
# calculamos gradiente 1D y luego lo mapeamos a rango uplift_range
if np.ptp(values) > 0 and len(values) == n_cells_dem:
    # interpretamos values como elevacion flattened y estimamos diferencias finite-diff
    diff = np.abs(np.diff(values, prepend=values[0]))
    uplift_vals = rescale(diff, uplift_range[0], uplift_range[1])
    uplift_flat = uplift_vals
else:
    # Caso general: reescala directamente el array a rango uplift
    uplift_vals = rescale(values, uplift_range[0], uplift_range[1])
    if len(uplift_vals) != n_cells_dem:
        uplift_flat = np.interp(np.arange(n_cells_dem),
                                np.linspace(0, n_cells_dem-1, num=len(uplift_vals)),
                                uplift_vals)
    else:
        uplift_flat = uplift_vals

pd.DataFrame(uplift_flat).to_csv("data/stepMapOficial.csv", index=False, header=False)
print("Guardado: stepMapOficial.csv (len={})".format(len(uplift_flat)))

# --- Sanity checks ---
print("Uplift (min,max) ->", np.nanmin(uplift_flat), np.nanmax(uplift_flat))

Input shape: (2365023, 1)
Valores válidos leídos: 2365023 min,max =  -100.0 15900.0
Celdas DEM: 2365023
Guardado: stepMapOficial.csv (len=2365023)
Uplift (min,max) -> 5e-05 0.0015


In [ ]:
import rasterio
import numpy as np
import pandas as pd

In [ ]:
with rasterio.open("dem_resampled.asc") as src:
    array = src.read(1)
    transform = src.transform

In [ ]:
rows, cols = np.where(array != -9999)
xs, ys = rasterio.transform.xy(transform, rows, cols)
elev = array[rows, cols]

In [ ]:
df = pd.DataFrame({
    "x": xs,
    "y": ys,
    "elev": elev
})

In [ ]:
df.to_csv("dem1_100m.csv", index=False)

In [ ]:
df = pd.read_csv("dem1_100m.csv", sep=",", engine="python")

In [ ]:
if len(df.columns) == 1:
    df = df.iloc[:,0].str.split(",", expand=True)

In [ ]:
df.to_csv("dem1_100m.csv", index=False)

In [ ]:
with open("dem1_100m.csv", "r") as f:
    text = f.read()

In [ ]:
text = text.replace(",", " ")

with open("dem_space_100m.txt", "w") as f:
    f.write(text)

In [ ]:
#Iniciando a programar

In [1]:
# Driver robusto para correr Badlands en saltos de 250 ka (compatible Python 3.5)
import time
import os
import glob
from badlands.model import Model as badlandsModel

# --- Parámetros de la corrida ---
xml = "input12.xml"
final_t = 20000000.0   # 20 Ma en años
step   = 250000.0      # 250 ka en años

# --- Cargar modelo (solo parsea) ---
model = badlandsModel()
model.load_xml(xml)

# --- Determinar tiempo inicial (usar model.tNow si existe, si no tomar model.input.tStart) ---
if hasattr(model, "tNow") and model.tNow is not None:
    cur = float(model.tNow)
else:
    cur = float(getattr(model.input, "tStart", 0.0))

print("Inicio de la corrida. Tiempo inicial (años):", cur, "  (%.3f Ma)" % (cur / 1e6))
print("Objetivo final (años):", final_t, "  (%.1f Ma)" % (final_t / 1e6))
print("Paso solicitado (años):", step, "  (%.0f ka)" % (step / 1000.0))
print("Salida escrita en (intento):", getattr(model.input, "outDir", "output"))

# función auxiliar para listar h5 en carpeta output/h5 (intenta varias rutas)
def list_h5(outdir=None):
    candidates = []
    if outdir:
        candidates.append(os.path.join(outdir, "h5"))
        candidates.append(outdir)
    # rutas habituales
    candidates.extend(["output/h5", "output_0/h5", "output/hdf5", "output"])
    found = []
    for c in candidates:
        if os.path.exists(c):
            found.extend(glob.glob(os.path.join(c, "*.hdf5")))
            found.extend(glob.glob(os.path.join(c, "*.h5")))
    return sorted(found)

# --- Loop controlador: llamar run_to_time con target absoluto (cur+step) ---
try:
    while cur < final_t:
        target = min(final_t, cur + step)
        t0 = time.time()
        print("\n[*] Ejecutando model.run_to_time({}) ...".format(int(target)))
        model.run_to_time(target)   # avanza desde donde quedó hasta 'target'
        dt = time.time() - t0

        # actualizar cur desde model.tNow (si está) o desde model.input.tNow si existiera
        cur = getattr(model, "tNow", None)
        if cur is None:
            cur = getattr(model.input, "tNow", None)
        if cur is None:
            # si por alguna razón ambos son None, estimamos por 'target'
            cur = target

        print("  -> Llegado a tNow = {} años ({} Ma). Paso tardó {:.1f} s".format(int(cur), cur/1e6, dt))

        # listar archivos h5 para ver si se escribieron
        h5list = list_h5(getattr(model.input, "outDir", None))
        print("  -> archivos h5 encontrados (ej.):", len(h5list))
        if len(h5list) > 0:
            # mostramos los últimos 3 si hay
            for f in h5list[-3:]:
                try:
                    print("     -", os.path.basename(f), "size(MB)={:.1f}".format(os.path.getsize(f)/1e6))
                except:
                    print("     -", os.path.basename(f))
        # seguir al siguiente paso
except KeyboardInterrupt:
    print("\n==> CORRIDA interrumpida por el usuario. Último tNow =", getattr(model, "tNow", None))

print("\nScript finalizado. Tiempo final registrado model.tNow =", getattr(model, "tNow", None))

KeyboardInterrupt: 

In [ ]:
from badlands.model import Model
import time

model = Model()
model.load_xml("input12.xml")

cur = model.tNow if model.tNow is not None else model.input.tStart
final_t = 20000000.0
step = 250000.0

while cur < final_t:
    target = min(final_t, cur + step)
    print("[*] Ejecutando model.run_to_time({}) ...".format(int(target)))
    model.run_to_time(target)
    cur = model.tNow if model.tNow is not None else target
    print(" -> Llegado a tNow = {} años ({:.2f} Ma)".format(int(cur), cur / 1e6))

[*] Ejecutando model.run_to_time(250000) ...
   - Writing outputs (12.37 seconds; tNow = 0.0)
tNow = 250000.0 (16.92 seconds)
   - Writing outputs (6.70 seconds; tNow = 250000.0)
 -> Llegado a tNow = 250000 años (0.25 Ma)
[*] Ejecutando model.run_to_time(500000) ...
tNow = 500000.0 (2.85 seconds)
   - Writing outputs (6.21 seconds; tNow = 500000.0)
 -> Llegado a tNow = 500000 años (0.50 Ma)
[*] Ejecutando model.run_to_time(750000) ...
tNow = 750000.0 (2.63 seconds)
   - Writing outputs (5.94 seconds; tNow = 750000.0)
 -> Llegado a tNow = 750000 años (0.75 Ma)
[*] Ejecutando model.run_to_time(1000000) ...
tNow = 1000000.0 (2.62 seconds)
   - Writing outputs (6.38 seconds; tNow = 1000000.0)
 -> Llegado a tNow = 1000000 años (1.00 Ma)
[*] Ejecutando model.run_to_time(1250000) ...
tNow = 1250000.0 (3.86 seconds)
   - Writing outputs (6.19 seconds; tNow = 1250000.0)
 -> Llegado a tNow = 1250000 años (1.25 Ma)
[*] Ejecutando model.run_to_time(1500000) ...
tNow = 1500000.0 (2.88 seconds)
   -

tNow = 11750000.0 (2.97 seconds)
   - Writing outputs (6.86 seconds; tNow = 11750000.0)
 -> Llegado a tNow = 11750000 años (11.75 Ma)
[*] Ejecutando model.run_to_time(12000000) ...
tNow = 12000000.0 (2.81 seconds)
   - Writing outputs (6.13 seconds; tNow = 12000000.0)
 -> Llegado a tNow = 12000000 años (12.00 Ma)
[*] Ejecutando model.run_to_time(12250000) ...
tNow = 12250000.0 (3.13 seconds)
   - Writing outputs (6.95 seconds; tNow = 12250000.0)
 -> Llegado a tNow = 12250000 años (12.25 Ma)
[*] Ejecutando model.run_to_time(12500000) ...
tNow = 12500000.0 (3.71 seconds)
   - Writing outputs (6.96 seconds; tNow = 12500000.0)
 -> Llegado a tNow = 12500000 años (12.50 Ma)
[*] Ejecutando model.run_to_time(12750000) ...
tNow = 12750000.0 (3.55 seconds)


In [ ]:
# modificar_intervals.py
import xml.etree.ElementTree as ET
import shutil

xml_in = "input12.xml"
xml_out = "input_forced_out.xml"
backup = "input_backup.xml"

# copia de seguridad
shutil.copyfile(xml_in, backup)
print("Backup creado:", backup)

tree = ET.parse(xml_in)
root = tree.getroot()

dt_output = "250000"  # 250 ka

changed = []

for elem in root.iter():
    taglow = elem.tag.lower()
    # Si el nodo tiene atributo 'interval' lo forzamos
    if 'interval' in elem.attrib:
        old = elem.attrib.get('interval')
        elem.set('interval', dt_output)
        changed.append((elem.tag, 'interval', old, dt_output))
    # Si el tag incluye 'timeseries' o 'timeSeries' o 'series' lo ajustamos o lo desactivamos
    if 'timeseries' in taglow or 'timeseries' in elem.tag.lower() or 'time' in taglow and 'series' in taglow:
        # intentamos forzar su intervalo si existe
        if 'interval' in elem.attrib:
            old = elem.attrib.get('interval')
            elem.set('interval', dt_output)
            changed.append((elem.tag, 'interval', old, dt_output))
        else:
            # si no tiene interval, agregamos un atributo interval con valor grande para evitar salidas frecuentes
            elem.set('interval', dt_output)
            changed.append((elem.tag, 'interval', None, dt_output))

# Escribir nuevo XML
tree.write(xml_out)
print("XML modificado escrito en:", xml_out)
print("Cambios realizados:")
for c in changed:
    print(" - Tag {} attribute {}: {} -> {}".format(*c))

In [ ]:
#ARREGLA ENCABEZADO Y COMAS POR ESPACIO EN EL CSV 

with open("data/uplift_step_interpolated_1D.csv", "r") as f:
    text = f.read()

# Reemplazar comas por espacios
text = text.replace(",", " ")

with open("uplift2_step_interpolated_1D.csv", "w") as f:
    f.write(text)

In [ ]:
#Arreglar grilla de archivos uplift y precip
#UNICAMENTE CON ASC O TIF

import rasterio
import numpy as np

# Ruta al DEM (puede ser .tif o .asc)
dem_path = "uplift_step_pichilemu_0-0003M.csv"  # o "dem.tif"
uplift_path = "data/uplift_direct.tif"
precip_path = "data/precip_direct2.tif"

# Abrir DEM como referencia
with rasterio.open(dem_path) as dem:
    profile = dem.profile
    dem_array = dem.read(1)
    print("✅ DEM cargado con shape:", dem_array.shape)

def raster_to_grid(raster_path, dem_profile):
    with rasterio.open(raster_path) as src:
        data = src.read(
            out_shape=(1, dem_profile["height"], dem_profile["width"]),
            resampling=rasterio.enums.Resampling.bilinear
        )[0]
    return data

uplift_grid = raster_to_grid(uplift_path, profile)
precip_grid = raster_to_grid(precip_path, profile)

print("✅ Uplift shape:", uplift_grid.shape)
print("✅ Precip shape:", precip_grid.shape)

# Guardar en CSV grilla (igual que DEM)
    # CSV con coma (útil para inspección)
np.savetxt("uplift_grilla.csv", uplift_grid, delimiter=",", fmt="%.6f")
np.savetxt("precip_grilla.csv", precip_grid, delimiter=",", fmt="%.6f")

    # TXT con espacios (formato Badlands)
np.savetxt("uplift_ready.txt", uplift_grid, delimiter=" ", fmt="%.6f")
np.savetxt("precip_ready.txt", precip_grid, delimiter=" ", fmt="%.6f")

print("✅ Archivos listos para Badlands: uplift_ready.txt y precip_ready.txt")

In [ ]:
#Adicional para cargar grillas

In [3]:
import pandas as pd

# === Leer uplift.csv detectando separador automáticamente ===
uplift_df = pd.read_csv("data/precipitation_map_1m.csv", sep=None, engine="python", header=None)

# === Verificar la estructura ===
print(uplift_df.head())

# === Si tienes tres columnas (x, y, uplift) ===
# Extraer solo la columna de uplift
uplift_values = uplift_df.iloc[:, 0]  # la última columna

# === Guardar CSV solo con valores de uplift, sin encabezado ni índice ===
uplift_values.to_csv("data/precipitation_map_1m.csv", index=False, header=False)


        0         1         2
0  precip         x         y
1     1.0  -11119.0  -13495.0
2     1.0  -11019.0  -13495.0
3     1.0  -10919.0  -13495.0
4     1.0  -10819.0  -13495.0


In [ ]:
#Grilla DEM = Grilla uplift y precip
#OBTENER UNA TIF UPL/PRECIP CON ASC O TIF

import pandas as pd
import numpy as np
import rasterio
from rasterio.transform import from_origin
from rasterio.crs import CRS

# Reabrir el DEM y asignar CRS manualmente
dem_path = "dem_resampled_100m.asc"
with rasterio.open(dem_path, "r+") as dem:
    dem.crs = CRS.from_epsg(32719)  # Asignamos EPSG:32719 manualmente
    print("✅ CRS asignado al DEM:", dem.crs)

# === 1. Cargar CSV de uplift ===
df = pd.read_csv("data/uplift_fixed.csv")

# Asegurar columnas correctas
if len(df.columns) == 1:
    df = df.iloc[:, 0].str.split(",", expand=True)
df.columns = ["x", "y", "z"]
df = df.apply(pd.to_numeric, errors="coerce").dropna()

# === 2. Cargar DEM como referencia ===
with rasterio.open("dem_resampled_100m.asc") as dem:
    nx, ny = dem.width, dem.height
    bounds = dem.bounds
    x_min, x_max = bounds.left, bounds.right
    y_min, y_max = bounds.bottom, bounds.top
    res_x = (x_max - x_min) / nx
    res_y = (y_max - y_min) / ny

# === 3. Crear matriz vacía ===
grid = np.full((ny, nx), -9999.0, dtype=np.float32)

# === 4. Asignar valores directamente a la grilla ===
for _, row in df.iterrows():
    col = int((row["x"] - x_min) / res_x)
    row_idx = int((y_max - row["y"]) / res_y)  # invertimos eje Y
    if 0 <= row_idx < ny and 0 <= col < nx:
        grid[row_idx, col] = row["z"]

# === 5. Guardar como GeoTIFF ===
transform = from_origin(x_min, y_max, res_x, res_y)
with rasterio.open(
    "data/uplift_direct.tif",
    "w",
    driver="GTiff",
    height=ny,
    width=nx,
    count=1,
    dtype=grid.dtype,
    crs="EPSG:32719",
    transform=transform,
    nodata=-9999
) as dst:
    dst.write(grid, 1)

print("✅ Archivo generado sin interpolación: uplift_direct.tif")
print("📐 Dimensiones:", grid.shape)

In [ ]:
#REORDENAR UPLIFT EN BASE AL CASO DEL CSV

import pandas as pd
import numpy as np
import os

# ---- Parámetros / nombres de archivo ----
dem_path = "pichilemu_xyz.csv"
step_path = "data/stepMapPich4.csv"

# ---- 1) Leer DEM y obtener grilla ----
dem = pd.read_csv(dem_path, sep=r"\s+", header=None, names=["x","y","elev"], engine='python')
x_unique = np.sort(dem["x"].unique())
y_unique = np.sort(dem["y"].unique())
ncols = len(x_unique)
nrows = len(y_unique)
expected = ncols * nrows

print("DEM: ncols={}, nrows={}, expected cells={}".format(ncols, nrows, expected))

# ---- 2) Leer columna de stepMap robustamente ----
def read_single_column(path):
    # Intentos variados para leer la columna que contenga los valores
    #  - ignora líneas comentadas con '#'
    #  - acepta separadores por espacios/tabs o comas
    try:
        df = pd.read_csv(path, sep=r"\s+", header=None, comment='#', engine='python')
        arr = df.values.flatten()
    except Exception:
        try:
            arr = np.loadtxt(path, comments='#')
        except Exception as e:
            raise RuntimeError("No pude leer {} : {}".format(path, e))
    # keep only finite numeric values
    arr = np.asarray(arr, dtype=float)
    arr = arr[np.isfinite(arr)]
    return arr

values = read_single_column(step_path)
print("Lectura stepMap: {} valores (sin NaN)".format(len(values)))

# ---- 3) Mapear según el caso ----
gx, gy = np.meshgrid(x_unique, y_unique)  # shape (nrows, ncols)

def save_grid(u_grid, outname):
    df = pd.DataFrame({
        "x": gx.flatten(),
        "y": gy.flatten(),
        "uplift": u_grid.flatten()
    })
    df.to_csv(outname, index=False)
    stats = (df.uplift.min(), df.uplift.mean(), df.uplift.max())
    print("Guardado: {}  (min,mean,max) = ({:.6g}, {:.6g}, {:.6g})".format(outname, stats[0], stats[1], stats[2]))

# Caso A: la longitud coincide exactamente con la grilla
if len(values) == expected:
    # Intento 1: orden C (x rápido, y lento) -> típico flatten C-order
    grid_C = values.reshape((nrows, ncols), order='C')
    save_grid(grid_C, "uplift_step_orderC.csv")

    # Intento 2: orden F (Fortran, y rápido, x lento) -> alternativa
    grid_F = values.reshape((nrows, ncols), order='F')
    save_grid(grid_F, "uplift_step_orderF.csv")

    print("He generado dos archivos: 'uplift_step_orderC.csv' y 'uplift_step_orderF.csv'.")
    print("Por defecto prueba primero 'uplift_step_orderC.csv' en tu input.xml; si la orientación espacial no coincide, usa la versión 'orderF'.")

# Caso B: un valor por FILA (len == nrows) -> se repite a lo largo de columnas
elif len(values) == nrows:
    grid = np.tile(values.reshape((nrows,1)), (1, ncols))
    save_grid(grid, "uplift_step_from_rows.csv")
    print("Interpreté stepMap como un valor por fila (y).")

# Caso C: un valor por COLUMNA (len == ncols) -> se repite a lo largo de filas
elif len(values) == ncols:
    grid = np.tile(values.reshape((1,ncols)), (nrows, 1))
    save_grid(grid, "uplift_step_from_cols.csv")
    print("Interpreté stepMap como un valor por columna (x).")

# Caso D: un único valor -> rellenar toda la grilla con ese valor
elif len(values) == 1:
    grid = np.full((nrows, ncols), values[0])
    save_grid(grid, "uplift_step_constant.csv")
    print("Step map contiene un único valor: creado uplift constante.")

# Caso E: longitud distinta, pero >1 -> interpolación 1D a lo largo del flattened index
elif 1 < len(values) < expected:
    # Interpolación simple 1D en el índice flattened
    src_idx = np.linspace(0, expected - 1, num=len(values))
    tgt_idx = np.arange(expected)
    interp_flat = np.interp(tgt_idx, src_idx, values)
    grid = interp_flat.reshape((nrows, ncols))
    save_grid(grid, "uplift_16000.csv")
    print("Se generó 'uplift_step_interpolated_1D.csv' por interpolación 1D del vector fuente.")

# Caso F: ninguno de los anteriores -> no se puede mapear automáticamente
else:
    raise RuntimeError("No pude mapear automaticamente: len(values)={} no coincide con expected/nrows/ncols = {}/{}/{}."
                       .format(len(values), expected, nrows, ncols))

In [1]:
#GENERAR PRECIP CON GRILLA DEM Y VALORES FIJOS

import pandas as pd
import numpy as np

# Cargar DEM
dem = pd.read_csv("xyz.csv", sep=r"\s+", header=None, names=["x", "y", "elev"], engine="python")

# Extraer coordenadas únicas
x_unique = np.sort(dem["x"].unique())
y_unique = np.sort(dem["y"].unique())

ncols = len(x_unique)
nrows = len(y_unique)
expected = ncols * nrows

precip_value = 1.0
precip_grid = np.full((nrows, ncols), precip_value)

gx, gy = np.meshgrid(x_unique, y_unique)  # orden C: x rápido, y lento

df_precip = pd.DataFrame({
    "x": gx.flatten(),
    "y": gy.flatten(),
    "precip": precip_grid.flatten()
})

df_precip.to_csv("data/precipitation_map_1m.csv", index=False)

In [ ]:
#GENERAR PRECIP CON GRILLA DEM Y VALORES GRADIENTES

import numpy as np
import pandas as pd

# === Parámetros espaciales del área de estudio ===
xmin, xmax = -11119.0, 27981.0
ymin, ymax = -13495.0, 61205.0
ncols, nrows = 1761, 1343  # ajustar a tu DEM

# === Crear malla espacial ===
x = np.linspace(xmin, xmax, ncols)
y = np.linspace(ymin, ymax, nrows)
xx, yy = np.meshgrid(x, y)

# === Gradiente de precipitación: costa (Oeste) → interior (Este) ===
# 5.0 en la costa, 1.0 en el interior
precip = np.interp(xx, [xmin, xmax], [5.0, 1.0])

# Añadir una leve variabilidad (±10%) para simular efecto topográfico
precip = precip * np.random.normal(1, 0.1, precip.shape)

# === Aplanar y exportar sin encabezado ===
precip_flat = precip.flatten()
pd.DataFrame(precip_flat).to_csv("precipitation_map_grad.csv", index=False, header=False)

print("✅ Archivo 'precipitation_map_grad.csv' creado con", len(precip_flat), "valores entre ~1.0 y 5.0.")